Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

Here's a brief markup that explains the purpose and usage of the `segments_dict`:

---

## Neuroimaging File Extraction Dictionary

The `segments_dict` is a predefined dictionary structured to facilitate the extraction of specific types of neuroimaging files. Each key in the dictionary represents a distinct neuroimaging segment, and its associated value is another dictionary containing the following fields:

- **path**: This should be filled with the absolute path to the base directory containing the neuroimaging files for the corresponding tissue segment. 
- **glob_name_pattern**: This is the string pattern that will be used to "glob" or search for the specific files within the provided path. It helps in identifying and extracting the desired files based on their naming conventions.

Here's a breakdown of the segments and their respective fields:

### 1. Cerebrospinal Fluid (CSF)
- **path**: Absolute path to the base directory containing CSF files.
- **glob_name_pattern**: File pattern to search for CSF files.

### 2. Grey Matter
- **path**: Absolute path to the base directory containing grey matter files.
- **glob_name_pattern**: File pattern to search for grey matter files.

### 3. White Matter
- **path**: Absolute path to the base directory containing white matter files.
- **glob_name_pattern**: File pattern to search for white matter files.

---

**Instructions**: Please fill out the `path` and `glob_name_pattern` fields for each segment in the `segments_dict`. This will ensure that the extraction process can locate and identify the appropriate neuroimaging files for further analysis.

---

Define Nifti Information

In [6]:
base_directory = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd'
grey_matter_glob_name_pattern = 'mri/mwp1*'
white_matter_glob_name_pattern = 'mri/mwp2*'
csf_glob_name_pattern = 'mri/mwp3*'

Downsample Niftis

In [7]:
import os
import glob
from nilearn import image as nli
from nilearn.image import resample_to_img
import nibabel as nib
import nibabel.processing
from tqdm import tqdm
from nimlab import datasets as nimds


def downsample_image(input_path, output_path):
    """
    Function to downsample a 3D image to a new voxel size using a target affine.
    
    Args:
    input_path (str): Filepath to the input image.
    output_path (str): Filepath to save the output image.
    target_voxel_size (list): Target voxels to resample to.
    """
    # Load the image
    img = nib.load(input_path)
    mni_mask = nimds.get_img("mni_icbm152")
    
    # Downsample the image using the target affine
    resampled_img = resample_to_img(img, mni_mask)

    # Save the downsampled image
    nib.save(resampled_img, output_path)
    
mni_mask = nimds.get_img("mni_icbm152")
mask_data = mni_mask.get_fdata().flatten()

def downsample_to_mni152_images_in_folder(input_folder_pattern, dry_run=True):
    """
    Function to downsample all 3D images in a folder to a new voxel size.
    
    Args:
    input_folder_pattern (str): Glob pattern to find the input images.
    target_voxel_size (list): Target voxels to resample to.
    """
    # Find all input image filepaths
    input_filepaths = glob.glob(input_folder_pattern)
    print('Will search:, ', input_folder_pattern)

    # Loop over each input image
    output_path_list = []
    for input_path in tqdm(input_filepaths):
        # Define the output path
        base, ext = os.path.splitext(input_path)
        if ext == '.gz':
            base, ext2 = os.path.splitext(base)
            ext = ext2 + ext
        output_path = base + '_resampled' + ext

        # Downsample the image
        if dry_run:
            pass
        else:
            downsample_image(input_path, output_path)
        output_path_list.append(output_path)
    
    return output_path_list


# Usage:
# downsample_images_in_folder('/path/to/your/images/*/*/anat/*mwp1*.nii', '/path/to/target/resolution/image.nii')

In [8]:
def downsample_images_to_mni152(base_directory, grey_matter_glob_name_pattern, white_matter_glob_name_pattern, csf_glob_name_pattern):
    """
    Downsamples images to MNI152 standard space and saves them in a specified directory.
    
    Parameters:
    - base_directory (str): The base directory where the images are located.
    - grey_matter_glob_name_pattern (str): Glob pattern for grey matter data.
    - white_matter_glob_name_pattern (str): Glob pattern for white matter data.
    - csf_glob_name_pattern (str): Glob pattern for cerebrospinal fluid data.
    
    Returns:
    - dict: A dictionary containing paths of the downsampled images for each segment.
    """
    
    segments_dict = {
        'grey_matter': {'path': base_directory, 'glob_name_pattern': grey_matter_glob_name_pattern},
        'white_matter': {'path': base_directory, 'glob_name_pattern': white_matter_glob_name_pattern},
        'cerebrospinal_fluid': {'path': base_directory, 'glob_name_pattern': csf_glob_name_pattern}
    }

    output_paths_dict = {}

    for k, v in segments_dict.items():
        output_paths_dict[k] = downsample_to_mni152_images_in_folder(os.path.join(v['path'], v['glob_name_pattern']), dry_run=False)
        print(f'Downsampled {k} segment data \n ')
        print(f'Saved files to {output_paths_dict[k]}')
        print('-------------------------------- \n')

    return output_paths_dict


In [9]:
output_paths = downsample_images_to_mni152(base_directory, grey_matter_glob_name_pattern, white_matter_glob_name_pattern, csf_glob_name_pattern)

Will search:,  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp1*


100%|██████████| 13/13 [00:07<00:00,  1.72it/s]


Downsampled grey_matter segment data 
 
Saved files to ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp17_Dicom_t1_mprage_cor_p2_iso_1.0_20210601163753_15_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp12_Dicom_COR_FSPGR_BRAVO_20151010124429_7_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp18_Dicom_t1_mprage_cor_p2_iso_1.0_20190219110928_8_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp16_Dicom_Cor_T1_SPGR_20160831131900_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp15_Dicom_MP-RAGE-NEW_20160226111830_4_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp13_Dicom_3D_COR_T1_MP-RAGE_20220714111911_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mr

100%|██████████| 13/13 [00:07<00:00,  1.75it/s]


Downsampled white_matter segment data 
 
Saved files to ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp25_Dicom_MP-RAGE-NEW_20160226111830_4_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp22_Dicom_COR_FSPGR_BRAVO_20151010124429_7_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp27_Dicom_t1_mprage_cor_p2_iso_1.0_20210601163753_15_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp28_Dicom_t1_mprage_cor_p2_iso_1.0_20190219110928_8_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp24_Dicom_MP-RAGE-NEW_20180205145012_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp23_Dicom_3D_COR_T1_MP-RAGE_20220714111911_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/m

100%|██████████| 13/13 [00:07<00:00,  1.76it/s]

Downsampled cerebrospinal_fluid segment data 
 
Saved files to ['/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp36_Dicom_Cor_T1_SPGR_20160831131900_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp312_Dicom_t2_spc_da-fl_sag_p2_iso_1.0_20161226140204_12_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp38_Dicom_t1_mprage_cor_p2_iso_1.0_20190219110928_8_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp311_Dicom_t1_space_ir_cor_p2_iso_20211203105200_9_real_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp33_Dicom_3D_COR_T1_MP-RAGE_20220714111911_5_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp32_Dicom_COR_FSPGR_BRAVO_20151010124429_7_Crop_1_resampled.nii', '/Users/cu135/Dropbox (Partners HealthCare)/

If you are running this off a cloud, make sure your files are available locally.

In [10]:
segments_dict = {
    'grey_matter': {'path': base_directory, 'glob_name_pattern': grey_matter_glob_name_pattern},
    'white_matter': {'path': base_directory, 'glob_name_pattern': white_matter_glob_name_pattern},
    'cerebrospinal_fluid': {'path': base_directory, 'glob_name_pattern': csf_glob_name_pattern}
}

output_paths_dict = {}

for k, v in segments_dict.items():
    output_paths_dict[k] = downsample_to_mni152_images_in_folder(os.path.join(v['path'], v['glob_name_pattern']), dry_run = False)
    print(f'Downsampled {k} segment data \n ')
    print(f'Saved files to {v}')
    print('-------------------------------- \n')

Will search:,  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp1*


100%|██████████| 26/26 [00:07<00:00,  3.44it/s]


Downsampled grey_matter segment data 
 
Saved files to {'path': '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd', 'glob_name_pattern': 'mri/mwp1*'}
-------------------------------- 

Will search:,  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp2*


100%|██████████| 26/26 [00:07<00:00,  3.50it/s]


Downsampled white_matter segment data 
 
Saved files to {'path': '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd', 'glob_name_pattern': 'mri/mwp2*'}
-------------------------------- 

Will search:,  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd/mri/mwp3*


100%|██████████| 26/26 [00:07<00:00,  3.47it/s]

Downsampled cerebrospinal_fluid segment data 
 
Saved files to {'path': '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/mgh_ftd', 'glob_name_pattern': 'mri/mwp3*'}
-------------------------------- 



Your files have now been downsampled to 2x2x2mm resolution.
Carry on to Notebook 02 to derive atrophy seeds

-Calvin